### Import lib cần thiết

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os 
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, eigs
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import torch
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

c:\Users\Kieu Trung Ha\Desktop\Key-Frames-Extraction-from-Video\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Generate caption for image

In [2]:
# load model 
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")

In [ ]:
# get list of folder keyframes to generate caption 
keyframes_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\keyframes\keyframes'
map_keyframes_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes'
folder_save_csv = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes-caption'

list_keyframes_path = os.listdir(keyframes_folder_path)
print(list_keyframes_path[120:123])

In [ ]:
# get list of folder keyframes to generate caption 
keyframes_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\keyframes\keyframes'
map_keyframes_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes'
folder_save_csv = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes-caption'

list_keyframes_path = os.listdir(keyframes_folder_path)

for folder in list_keyframes_path[120:123]: 
    list_image = os.listdir(os.path.join(keyframes_folder_path, folder))
    
    # khởi tại list caption do model generate ra 
    list_caption = []
    
    # generate caption 
    for img_name in list_image: 
        img_path = os.path.join(os.path.join(keyframes_folder_path, folder), img_name)
        
        # đọc ảnh và chuyển về rgb
        raw_image = Image.open(img_path).convert('RGB')
        
        inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)

        out = model.generate(**inputs)
        
        list_caption.append(processor.decode(out[0], skip_special_tokens=True))
        
    # đọc tệp csv thành 1 dataframe 
    df = pd.read_csv(os.path.join(map_keyframes_path, folder+'.csv'))
    
    # thêm cột caption vào dataframe
    df['Caption'] = list_caption

    # Lưu DataFrame vào tệp CSV đã cập nhật
    df.to_csv(folder+'_caption.csv', index=False)
    
    print(folder)

### Xử lý Duplicate keyframes 

In [4]:
# đầu vào là link của folder keyframes và tên của file map-keyframes
def filter_duplicate(keyframes_folder_path, map_keyframes_caption_folder_path):
    # get list name of images 
    list_img = os.listdir(keyframes_folder_path)

    # khởi tạo mảng trống để lưu trữ biểu đồ histogram 
    arr = np.empty((0, 1944), int)

    # khởi tạo dictionary để lưu trữ khung hình dưới dạng mảng 
    D = dict()

    for index, i in enumerate(list_img): 
        img_bgr = cv2.imread(os.path.join(keyframes_folder_path, i))
        # convert image from BGR to RGB 
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
        # lưu trữ ảnh dưới dạng mảng 
        D[index] = img_rgb
        
        # chia khung hình thành các khối 3*3 tổng có 9 blocks
        height, width, channels = img_rgb.shape
        
        # kích thước của mỗi block
        if height % 3 == 0:
            h_chunk = int(height/3)
        else:
            h_chunk = int(height/3) + 1

        if width % 3 == 0:
            w_chunk = int(width/3)
        else:
            w_chunk = int(width/3) + 1
            
        h=0
        w=0 
        feature_vector = []
        for a in range(1,4):
            h_window = h_chunk*a
            for b in range(1,4):
                frame = img_rgb[h : h_window, w : w_chunk*b , :]
                """
                    tìm histogram cho ảnh, [0, 1, 2] là danh sách  các kênh màu mà histogram được tính toán 
                    None: giá trị mask 
                    [6, 6, 6] là kích thước của histogram, tức là mỗi kênh màu, histogram sẽ tính toán 6 bin 
                    [0, 256, 0, 256, 0, 256] đây là phạm vi giá trị của các kênh màu
                """
                hist = cv2.calcHist(frame, [0, 1, 2], None, [6, 6, 6], [0, 256, 0, 256, 0, 256])  
                hist1= hist.flatten()  # flatten the hist to one-dimensinal vector 
                feature_vector += list(hist1)
                w = w_chunk*b
                
            h = h_chunk*a
            w= 0
        
        arr =np.vstack((arr, feature_vector ))
        
    final_arr = arr.transpose() #transposing so that i will have all frames in columns i.e M*N dimensional matrix 
    #where M is 1944 and N is number of frames
    
    # hàm csc_matrix là một hàm trong thư viện scipy sử dụng để tạo 1 ma trận trong định dạng Compressed Sparse Column 
    # định dạng CSC là một định dạng lưu trữ ma trận thưa (sparse matrix) trong đó các phần tử khác không được lưu trữ kèm index của chúng
    A = csc_matrix(final_arr, dtype=float)

    # svds là hàm dùng để phân tích giá trị riêng (singular value decomposition - SVD) trên ma trận thưa (sparse matrix), giúp phân tích một ma trận thành các thành phần như giá trị riêng, vector riêng và các giá trị đặc trưng khác 
    # top 63 singular values from 76082 to 508
    # u là ma trận các vector riêng trái 
    # s là một mảng chứa các giá trị riêng 
    # vt là ma trận chứa các vector riêng phải 
    u, s, vt = svds(A, k = 63)

    # trả về ma trận chuyển vị của vt
    v1_t = vt.transpose()

    # np.diag(s) tạo ra 1 ma trận đường chéo cho mảng s, các giá trị nằm trên đường chéo bằng với s, còn lại bằng 0 
    # các cột trong ma trận vt là các dữ liệu histogram được chiếu lên cơ sở trực giao
    projections = v1_t @ np.diag(s) 
    #formed by vectors of the left singular matrix u .The coordinates of the frames in this space are given by v1_t @ np.diag(s)
    #So we can see that , now we need only 63 dimensions to represent each column/frame 
    
    f = projections 

    # khởi tạo 1 dictionary lưu trữ các frames trong respective cluster 
    C = dict()

    # khởi tạo list các index của keyframes trùng nhau trong 1 scence
    list_keyframes = []

    # tạo ra 1 dictionary chứa 236 mảng kích thước 63 
    for i in range(f.shape[0]): 
        C[i] = np.empty((0, 63), int)
        
    # khởi tạo frame đầu tiên trong cụm đầu tiên 
    C[0] = np.vstack((C[0], f[0]))
    # C[0] = np.vstack((C[0], f[1]))

    # khởi tạo dictionary để lưu trữ centroids của mỗi cluster 
    E = dict()
    for i in range(projections.shape[0]):
        E[i] = np.empty((0, 63), int)
        
    E[0] = np.mean(C[0], axis=0)

    count = 0 
    for i in range(1, f.shape[0]):
        similarity = np.dot(f[i], E[count])/( (np.dot(f[i],f[i]) **.5) * (np.dot(E[count], E[count]) ** .5))
        
        if similarity < 0.9: 
            count+=1
            C[count] = np.vstack((C[count], f[i]))
            E[count] = np.mean(C[count], axis=0)   
        else: 
            C[count] = np.vstack((C[count], f[i])) 
            E[count] = np.mean(C[count], axis=0)  
            list_keyframes.append(i)

    # xóa file keyframe và row chứa keyframes trùng nhau
    kfcaption_dataframe = pd.read_csv(map_keyframes_caption_folder_path)
    
    for id, file_path in enumerate(os.listdir(keyframes_folder_path)):
        if id in list_keyframes:
            # xóa keyframe trong folder keyframes
            os.remove(os.path.join(keyframes_folder_path, file_path))

    
    # xóa row keyframes bị trùng 
    kfcaption_dataframe = kfcaption_dataframe.drop(list_keyframes).reset_index(drop=True)
    
    # thêm cột độ dài của sence 
    kfcaption_dataframe['length sence'] = kfcaption_dataframe['frame_idx'].diff().shift(-1)
    
    kfcaption_dataframe['Video'] = keyframes_folder_path.split('\\')[-1]
    
    kfcaption_dataframe.to_csv(map_keyframes_caption_folder_path, index=False) 
    
    

In [57]:
# lọc keyframes trùng nhau và xóa row cho tất cả các folder keyframes, sau đó thêm 1 cột tên video và cột chứa độ dài frame của keyframes đó
keyframes_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\keyframes'
map_keyframes_caption_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes-caption'

list_keyframes_folder = os.listdir(keyframes_folder_path)

for folder in list_keyframes_folder:
    folder_path = os.path.join(keyframes_folder_path, folder)
    file_mapkf_caption_path = os.path.join(map_keyframes_caption_path, folder + '_caption.csv')
    
    filter_duplicate(folder_path, file_mapkf_caption_path)

In [59]:
# nối tất cả file csv vào 1 file duy nhất
map_keyframes_caption_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes-caption'

# Danh sách để lưu các DataFrame từ các tệp CSV
dataframes = []

# Lặp qua từng tệp CSV
for file in os.listdir(map_keyframes_caption_path):
    # Đọc tệp CSV vào DataFrame
    df = pd.read_csv(os.path.join(map_keyframes_caption_path, file))
    
    # Xóa cột "n"
    df = df.drop("n", axis=1)
    
    # Thêm DataFrame vào danh sách
    dataframes.append(df)
    
# Nối các DataFrame thành một DataFrame duy nhất
merged_df = pd.concat(dataframes, ignore_index=True)

# Ghi DataFrame duy nhất vào tệp CSV
merged_df.to_csv('summary_map_keyframes.csv', index=False)

#### Đổi tên các file 

In [15]:
map_keyframes_caption_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\map-keyframes-caption'

list_mapkf_cap_path = os.listdir(map_keyframes_caption_path)

# đổi tên các file cùng định dạng
for file_path in list_mapkf_cap_path:
    old_name = os.path.join(map_keyframes_caption_path, file_path)
    
    if '_caption' not in file_path: 
        new_name = os.path.join(map_keyframes_caption_path, file_path.split('.')[0] + '_caption.csv')

        # Sử dụng hàm rename() để đổi tên file
        os.rename(old_name, new_name)

### Truy xuất 

In [71]:
# thêm 1 column chứa tên các img tương ứng với row 
summary_map_kf = pd.read_csv('summary_map_keyframes.csv')

list_image_name = []

# lặp qua tất cả các folder keyframes lấy ra tên của ảnh sau đó thêm vào list tên image
keyframes_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\keyframes'
for folder in os.listdir(keyframes_folder_path):
    list_img_name = os.listdir(os.path.join(keyframes_folder_path, folder))
    list_image_name.append(list_img_name)
    
flattened_list = [item for sublist in list_image_name for item in sublist]
print(len(flattened_list))

# tạo thêm 1 columns chứa tên ảnh 
summary_map_kf['img_name'] = flattened_list

summary_map_kf.to_csv('summary_map_keyframes_final.csv', index=False)

73044


In [60]:
import nltk
nltk.download('stopwords')  # Tải xuống tài nguyên stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Kieu Trung
[nltk_data]     Ha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Kieu Trung
[nltk_data]     Ha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import os
import pandas as pd
import cv2 
import csv 

# Thời điểm bắt đầu
start_time = time.time()

def preprocess_text(text):
    # Tách từ và loại bỏ stop words
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return tokens

def calculate_cosine_similarity(text1, text2):
    # Tiền xử lý và biểu diễn hai đoạn văn bản thành các vectơ
    tokens1 = preprocess_text(text1)
    tokens2 = preprocess_text(text2)
    all_tokens = list(set(tokens1 + tokens2))
    vector1 = [tokens1.count(token) for token in all_tokens]
    vector2 = [tokens2.count(token) for token in all_tokens]

    # Tính toán cosine similarity
    dot_product = sum(i*j for i, j in zip(vector1, vector2))
    magnitude1 = sum(i**2 for i in vector1) ** 0.5
    magnitude2 = sum(i**2 for i in vector2) ** 0.5
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    return cosine_similarity

def cut_video(input_path, output_path, start_frame, end_frame):
    # Tính toán thời gian bắt đầu và kết thúc dựa trên số khung hình trên giây
    duration = end_frame - start_frame
    start_time = start_frame / 25
    end_time = end_frame / 25

    # Cắt video con từ video ban đầu
    ffmpeg_extract_subclip(input_path, start_time, end_time, targetname=output_path)

def query(input_text, path_csv, keyframes_folder_path, video_folder_path, results_folder_path, time_dp=False) :
    df = pd.read_csv(path_csv)
    cosine_sim = []
    # Duyệt qua từng tệp tin trong thư mục
    for cap in df['Caption'] :
        cosine_sim.append(calculate_cosine_similarity(input_text, cap))

    # Sử dụng list comprehension để tạo ra một list các cặp (giá trị, số thứ tự)
    value_index_pairs = [(value, index) for index, value in enumerate(cosine_sim)]

    # Sắp xếp list các cặp theo giá trị giảm dần
    value_index_pairs.sort(reverse=True)
    
    # lấy ra list path của video và số frame sence tương ứng
    rs = []
    
    # In ra số thứ tự của 10 giá trị lớn nhất
    for value, index in value_index_pairs[:10]:
        # print("Giá trị:", value, " - Số thứ tự:", index , " Video :" ,  df.loc[index, 'Video'], " Frame :" , df.loc[index, 'frame_idx'])
        img_name = df.loc[index, 'img_name']
        folder_name = df.loc[index, 'Video']
        leng_sence = df.loc[index, 'length sence']
        img_path = os.path.join(keyframes_folder_path, os.path.join(folder_name, img_name))
        # result 
        result = []
        result.append(folder_name)
        result.append(leng_sence)
        result.append(df.loc[index, 'frame_idx'])
        result.append(img_name)
        rs.append(result)
        
        # print(img_path)
        # img = cv2.imread(img_path)
        # cv2.imshow(img_name, img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

    if time_dp == True: 
        # Thời điểm kết thúc
        end_time = time.time()
        # Tính thời gian chạy
        execution_time = end_time - start_time
        print("Thời gian chạy:", execution_time, "giây")
        
    # cut video và lưu vào folder kết quả
    for id, result in enumerate(rs):
        input_path = os.path.join(video_folder_path, result[0]+'.mp4')
        start_keyframe = int(result[2])
        leng_sence = int(result[1]) 
        end_keyframe = int(start_keyframe + leng_sence)
        output_path = os.path.join(results_folder_path, 'rank_'+str(id)+'_'+result[0]+'.mp4')
        
        cut_video(input_path, output_path, start_keyframe, end_keyframe)
    
        result_file_path = r'result.csv'
        # kiểm tra xem tệp tin csv tồn tại hay không 
        if not os.path.isfile(result_file_path): 
            # nếu tập tin không tồn tại, tạo tập tin mới 
            with open(result_file_path, 'w', newline='') as file: 
                writer = csv.writer(file)
                # thêm các tiêu đề cầ thiết 
                writer.writerow(['query', 'video', 'keyframe', 'length', 'rank'])
                
        with open(result_file_path, 'a', newline='') as file: 
            writer = csv.writer(file)
            # thêm các kết quả mới vào dòng cuối cùng 
            writer.writerow([input_text, result[0], result[3], result[1], id])
    
    # return rs

In [6]:
video_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\video\video'
results_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\Key-Frames-Extraction-from-Video\results'
keyframes_folder_path = r'C:\Users\Kieu Trung Ha\Desktop\AI Challenge\data_raw\keyframes'
path_csv = r'summary_map_keyframes_final.csv'

result = query("dairy cows are eating", path_csv, keyframes_folder_path, video_folder_path, results_folder_path, time_dp=False)
print(result)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
None


In [60]:
# clear folder video kết quả 
folder_path = r'C:\Users\Kieu Trung Ha\Desktop\Key-Frames-Extraction-from-Video\results'  # Đường dẫn đến thư mục của bạn

# Lặp qua tất cả các tệp tin trong thư mục và xóa chúng
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)